In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jan 11 13:54:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -rf darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.02 MiB | 18.86 MiB/s, done.
Resolving deltas: 100% (10353/10353), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-11 13:56:02--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  11.4MB/s    in 10s     

2022-01-11 13:56:13 (15.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3.cfg
!sed -i 's/width=416/width=192/' cfg/yolov3.cfg
!sed -i 's/height=416/height=192/' cfg/yolov3.cfg

!sed -i '610 s@classes=80@classes=6@' cfg/yolov3.cfg
!sed -i '696 s@classes=80@classes=6@' cfg/yolov3.cfg
!sed -i '783 s@classes=80@classes=6@' cfg/yolov3.cfg
!sed -i '603 s@filters=255@filters=33@' cfg/yolov3.cfg
!sed -i '689 s@filters=255@filters=33@' cfg/yolov3.cfg
!sed -i '776 s@filters=255@filters=33@' cfg/yolov3.cfg

In [ ]:
!echo -e '1\n2\n3\n4\n5\n6' > data/obj.names
!echo -e 'classes=6\ntrain=data/train.txt\nvalid=data/valid.txt\nnames=data/obj.names\nbackup=/content/drive/MyDrive/yolo/backup' > data/obj.data
!mkdir data/obj
!mkdir data/obj/train
!mkdir data/obj/valid

In [ ]:
!unzip "/content/drive/MyDrive/yolo/dataset/train.zip" -d data/obj/train
!unzip "/content/drive/MyDrive/yolo/dataset/valid.zip" -d data/obj/valid

Archive:  /content/drive/MyDrive/yolo/dataset/train.zip
  inflating: data/obj/train/IMG_20211115_220628.jpg  
  inflating: data/obj/train/IMG_20211115_222012.jpg  
  inflating: data/obj/train/IMG_20211115_220736.jpg  
  inflating: data/obj/train/IMG_20211115_222039.jpg  
  inflating: data/obj/train/IMG_20211115_220610.jpg  
  inflating: data/obj/train/IMG_20211115_222228.jpg  
  inflating: data/obj/train/photo5219811615145310170.jpg  
  inflating: data/obj/train/photo5219811615145310214.jpg  
  inflating: data/obj/train/IMG_20211115_221038.jpg  
  inflating: data/obj/train/5.jpg    
  inflating: data/obj/train/photo5219811615145310206.jpg  
  inflating: data/obj/train/photo5219811615145310188.jpg  
  inflating: data/obj/train/photo5219811615145310171.jpg  
  inflating: data/obj/train/IMG_20211115_220436.jpg  
  inflating: data/obj/train/photo5219811615145310196.jpg  
  inflating: data/obj/train/6.jpg    
  inflating: data/obj/train/13.jpg   
  inflating: data/obj/train/IMG_20211115_221

In [ ]:
labels = open("/content/drive/MyDrive/yolo/data/dice.names").read().strip().split("\n")
labels.index("1")

0

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re
import xml.etree.ElementTree as ET

def convert_coordinates(img_w, img_h, xmin, xmax, ymin, ymax):
  w = (xmax - xmin) / img_w
  h = (ymax - ymin) / img_h
  x = xmin / img_w + w / 2
  y = ymin / img_h + h / 2
  return x, y, w, h

def make_dataset(name):
  xml_file_paths = glob.glob(r"data/obj/" + name + "/*.xml")
  for i, file_path in enumerate(xml_file_paths):
      tree = ET.parse(file_path)
      text_converted = []
      root = tree.getroot()
      size = root.find('size')
      img_w = int(size.find('width').text)
      img_h = int(size.find('height').text)
      objects = root.findall('object')
      for obj in objects:
          class_name = obj.find('name').text
          class_id = labels.index(class_name)
          bndbox = obj.find('bndbox')
          xmin = int(bndbox.find('xmin').text)
          xmax = int(bndbox.find('xmax').text)
          ymin = int(bndbox.find('ymin').text)
          ymax = int(bndbox.find('ymax').text)
          x, y, w, h = convert_coordinates(img_w, img_h, xmin, xmax, ymin, ymax)
          text = "{} {} {} {} {}".format(class_id, x, y, w, h)
          text_converted.append(text)
      with open(file_path.replace('.xml', '.txt'), 'w') as fp:
          for item in text_converted:
              fp.writelines("%s\n" % item)
  images_list = glob.glob("data/obj/" + name + "/*.jpg")
  file = open("data/" + name + ".txt", "w")
  file.write("\n".join(images_list)) 
  file.close()

make_dataset('train')
make_dataset('valid')

In [ ]:
!ls "data"

9k.tree     eagle.jpg	 imagenet.labels.list	   obj.data	     scream.jpg
coco9k.map  giraffe.jpg  imagenet.shortnames.list  obj.names	     train.txt
coco.names  goal.txt	 labels			   openimages.names  valid.txt
dog.jpg     horses.jpg	 obj			   person.jpg	     voc.names


In [ ]:
# darknet53.conv.74
# /content/drive/MyDrive/yolo/backup/yolov3_last.weights

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3.cfg /content/drive/MyDrive/yolo/backup/yolov3_last.weights >> /content/drive/MyDrive/yolo/log/train.log -dont_show -map

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.857172), count: 6, class_loss = 0.001508, iou_loss = 0.084562, total_loss = 0.086071 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.758905), count: 14, class_loss = 0.719648, iou_loss = 0.875073, total_loss = 1.594721 
 total_bbox = 735187, rewritten_bbox = 0.005305 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.850157), count: 1, class_loss = 0.000049, iou_loss = 0.010902, total_loss = 0.010951 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.883374), count: 14, class_loss = 0.046028, iou_loss = 0.118764, total_loss = 0.164792 
 total_bbox = 735202, rewritten_bbox =

In [ ]:
# Download files after training to save the status
from google.colab import files
files.download("chart.png")
files.download("/content/drive/MyDrive/yolo/cfg/yolov3_dice.cfg")
files.download("/content/drive/MyDrive/yolo/backup/yolov3_last.weights")
files.download("/content/drive/MyDrive/yolo/backup/yolov3_best.weights")
files.download("/content/drive/MyDrive/yolo/dataset/test.zip")